# Tarefa 2: Implantar um modelo para inferência em tempo real

## Tarefa 2.1: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies
import boto3
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Examine o conjunto de dados processado de clientes.

In [ ]:
#explore-dataset
column_list = ['income','age','workclass','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.dtypes
lab_test_data.head()

Salve o modelo do laboratório de treinamento e ajuste no bucket-padrão do Amazon Simple Storage Service (Amazon S3). Configure um modelo usando **create_model** e configure **ModelDataUrl** para referenciar o modelo treinado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## Tarefa 2.2: Criar um endpoint por meio do modelo retreinado e sintetizado fornecido

Há três etapas para criar um endpoint usando o Amazon SageMaker SDK for Python:
1. Criar um modelo do SageMaker.
2. Criar uma configuração de endpoint para um endpoint HTTPS.
3. Criar um endpoint HTTPS.

Consulte [Criar o endpoint e implantar o modelo](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) para saber mais sobre a criação de endpoints.

Você já criou um modelo. Agora você já pode criar uma configuração de endpoint e um endpoint. 

Primeiro, defina o nome da configuração de endpoint e o tipo de instância que deseja usar. Depois, chame a API CreateEndpointConfig.

Para criar uma configuração de endpoint, você precisa definir as seguintes opções:
- **VariantName**: o nome da variante de produção (um ou mais modelos em produção).
- **ModelName**: o nome do modelo que você deseja hospedar. Esse é o nome que você especificou quando criou o modelo.
- **InstanceType**: o tipo de instância de computação.
- **InitialInstanceCount**: o número de instâncias a serem iniciadas primeiramente.

Para registrar as entradas no endpoint e as saídas de inferência dos endpoints em tempo real do SageMaker para o Amazon S3, ative um recurso chamado Data Capture. O Data Capture é muito usado para registrar informações que podem ser usadas para treinamento, depuração e monitoramento. Ao explorar o endpoint no Amazon SageMaker Studio, você verá mais detalhes sobre o endpoint quando o Data Capture for ativado. A configuração do Data Capture será apresentada depois neste laboratório para mostrar como ativá-lo.

Confira [Capture Data](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html) para saber mais sobre a adição do Data Capture.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date  
# so it you can search endpoints based on creation time.
endpoint_config_name = 'income-model-real-time-endpoint-{}'.format(create_date)                              
instance_type = 'ml.m5.xlarge'   
initial_sampling_percentage = 25 # Choose a value between 0 and 100
capture_modes = [ "Input",  "Output" ] # Specify input, output, or both

endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    DataCaptureConfig= {
        'EnableCapture': True, # Whether data should be captured or not.
        'InitialSamplingPercentage' : initial_sampling_percentage,
        'DestinationS3Uri': f's3://{bucket}/data-capture',
        'CaptureOptions': [{"CaptureMode" : capture_mode} for capture_mode in capture_modes]
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

Em seguida, crie um endpoint. Quando você cria um endpoint em tempo real, o SageMaker inicia instâncias de computação de machine learning (ML) e implanta um ou mais modelos de acordo com as especificações na configuração. Neste laboratório, você vai implantar somente um modelo para inferência. No SageMaker, é possível criar um endpoint multimodelo. Consulte [Invocar um endpoint multimodelo](https://docs.aws.amazon.com/sagemaker/latest/dg/invoke-multi-model-endpoint.html) para saber mais sobre os endpoints multimodelo.

Quando o endpoint estiver em serviço, a função auxiliar imprimirá o Amazon Resource Name (ARN) do endpoint. A criação do endpoint exigirá cerca de três a sete minutos.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


No SageMaker Studio, você pode examinar os detalhes do endpoint na aba **Endpoints**.

A próxima etapa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **real_time_inference.ipynb** para a lateral ou selecione (clique com o botão direito) a aba **real_time_inference.ipynb** e escolha **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o endpoint.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar o endpoint, retorne ao notebook a aba **real_time_inference.ipynb**.

1. Selecione o ícone **Página inicial do SageMaker**.
2. Escolha **Deployments** (Implantações).
3. Escolha **Endpoints**.

O SageMaker Studio exibe a aba **Endpoints**.

4. Selecione o endpoint que tem **income-model-real-time-** na coluna **Name** (Nome).

Se o endpoint não aparecer, selecione o ícone de atualização até que o endpoint apareça na lista.

O SageMaker Studio exibe a aba **ENDPOINT DETAILS** (DETALHES DO ENDPOINT).

5. Selecione a aba **AWS settings** (Configurações da AWS).

Se você abriu o endpoint antes que a criação fosse concluída, selecione o ícone de atualização até que o **Endpoint status** (Status do endpoint) seja alterado de *Creating* para *InService*.

O **Endpoint type** (Tipo de endpoint) está listado como **Real-time** (Em tempo real). As seções **Data capture settings** (Configurações do Data Capture) e **Endpoint configuration settings** (Definições de configuração do endpoint) mostram as configurações que você já escolheu no notebook.

## Tarefa 2.3: Invocar um endpoint para uma inferência em tempo real com registros de clientes em tempo real

Depois de implantar o modelo usando os serviços de host do SageMaker, você poderá testar o modelo nesse endpoint enviando dados a ele.

Há vários registros de cliente com uma receita igual ou superior a USD 50 mil (valor de **income** igual **1**) e vários outros com uma receita inferior a USD 50 mil (valor de **income** igual a **0**). Invoque o endpoint com esses registros e visualize as pontuações retornadas.

Para visualizar as previsões em tempo real do endpoint, leia o texto do corpo da mensagem retornado, que contém uma lista de pontuações de previsão. A pontuação de cada registro varia de **0** a **1**, com os números próximos de **1** indicando que esses clientes têm maior probabilidade de ter uma receita igual ou superior a USD 50 mil. Por exemplo, um cliente com a pontuação de previsão igual a **0,42** tem maior probabilidade de ter uma receita igual ou superior a USD 50 mil do que um cliente com a pontuação de previsão igual a **0,14**.

In [ ]:
#invoke-endpoint-real-time-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('56,3,6,6,0,3,1,0,0,1,0,13\n' +
                '29,2,2,2,0,1,0,0,0,0,0,70\n' +
                '79,0,1,1,0,3,5,0,0,0,0,20\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,3,2,0,0,0,0,32\n' +
                '31,0,1,1,2,1,2,1,1,0,0,40\n' +
                '23,0,1,1,1,0,1,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## Tarefa 2.4: Excluir o endpoint

A limpeza do endpoint pode ser realizada em três etapas. Primeiro, exclua o endpoint. Depois, exclua a configuração do endpoint. Por fim, se você não precisar mais do modelo implantado, exclua-o.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusão

Parabéns! Você usou o SageMaker para criar um endpoint em tempo real usando o SDK Python do SageMaker e para invocar o endpoint com sucesso.

A próxima tarefa do laboratório se concentra na implantação de um modelo para inferência usando a inferência sem servidor.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue na **Tarefa 3: Implantar um modelo para inferência sem servidor**.